In [39]:
from jax import jit, vmap, random
import jax.numpy as np
import numpy as onp
import jax

# enable float 64
from jax.config import config
config.update('jax_enable_x64', True)

In [49]:
# https://github.com/google/jax/issues/10144

def transition(xi, τ, dt=1):
    kb    = np.maximum(1.0 - np.exp(-τ*dt), 0)
    pop   = onp.random.binomial(xi, kb )
    return jax.device_put(pop)



DeviceArray([0, 0, 2, 3, 3], dtype=int64)

In [51]:
def transition(xi, τ, dt=1):
    kb    = np.maximum(1.0 - np.exp(-τ*dt), 0)
    pop   = onp.random.binomial(xi, kb )
    return jax.device_put(pop)


def sir(x, dt, β, γ, N):
    """ SIR model

    Args:
        x[t]:  State space at time t.
        dt:    Recovery rate.
        β:     Contact rate.
        γ:     Recovery rate.
        N:     Population.

    Returns:
        x[t+1]: State space at time t+1.
    """
    S, I, R, C = x

    s2i = transition(S, β * I / N *dt)
    i2r = transition(I, γ * dt)

    dS = -s2i
    dI = s2i - i2r
    dR = i2r
    C  = s2i

    return np.array([dS, dI, dR])
